In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd
import scipy

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers.experimental.preprocessing import Normalization
from tensorflow.keras import layers
import joblib

In [ ]:
import annutils

In [ ]:
import hvplot.pandas

# Load data

In [ ]:
dfin_on = pd.read_csv('smscg_input_on.csv',index_col=0,parse_dates=True)
dfin_off = pd.read_csv('smscg_input_off.csv',index_col=0,parse_dates=True)

dfout_on = pd.read_csv('smscg_output_on.csv',index_col=0,parse_dates=True)
dfout_off = pd.read_csv('smscg_output_off.csv',index_col=0,parse_dates=True)

# Create functions to load and interact with ANN model

In [ ]:
import annutils
import joblib
class ANNModel:
    '''
    model consists of the model file + the scaling of inputs and outputs
    '''
    def __init__(self,model,xscaler,yscaler):
        self.model=model
        self.xscaler=xscaler
        self.yscaler=yscaler
    def predict(self, dfin):
        return annutils.predict(self.model,dfin,self.xscaler,self.yscaler)
def load_model(location):
    '''
    load location i.e. RSAN018
    '''
    model=keras.models.load_model('%s_EC_ff_8x2.h5'%location)
    xscaler,yscaler=joblib.load('%s_EC-xyscaler.dump'%location)
    return ANNModel(model,xscaler,yscaler)

## Predict with loaded anns and scaler the values

In [ ]:
model=load_model('RSAN018')
dfp=model.predict(dfin_on)
pd.concat([dfout_on['RSAN018_EC'],dfp],axis=1).hvplot()

# For example increasing the exports by 100 cfs ...

In [ ]:
dfin_exp_plus100=dfin_on.copy()
dfin_exp_plus100.loc[:,'exports']+=100
dfin_exp_plus100

# ... what is the SAC flow adjustment needed to get the same predicted values?

In [ ]:
dfpexp=model.predict(dfin_exp_plus100)

In [ ]:
dfp.hvplot(label='original')*dfpexp.hvplot(label='+100 exports')

# Define the cost function
The first step to optimizing is to define a cost function which will be "minimized" by scipy.optimize

In this case the cost function takes an array of sac_flow_adjustment which is is added to the 'sac' column of the input. The sac_flow_adjustment is the same size as the dfin. 

Note: Be careful because the size of the problem i.e. the size of dfin can cause out of memory issues and can make the problem too hard to do on a single processor


In [ ]:
from sklearn.metrics import mean_squared_error
import scipy.optimize as optimize
def adjust_input(input_change,input_col,dfin):
    '''
    adjust input by adding the input_change array to the input_col of the pandas data frame dfin
    return a copy of the changed array 
    
    dfin is assumed to be the same size as input_change
    '''
    dfinc=dfin.copy()
    dfinc.loc[:,input_col]+=input_change
    return dfinc

def adjust_predict(model, input_change, input_col, dfin):
    '''
    adjust the dfin[:, input_col] by adding input_change
    returns prediction with the model 
    '''
    dfp=model.predict(adjust_input(input_change,input_col,dfin))
    return dfp

def cost_func(sac_flow_adjustment,model,dfin,dftarget):
    '''
    Calculate the SSE between the dforiginal and the model prediction using dfin as input. 
    The goal is to minimize this using scipy.optimize(cost_func)
    '''
    dfp=adjust_predict(model,sac_flow_adjustment,'sac',dfin)
    x,y=annutils.synchronize(dftarget,dfp)
    return mean_squared_error(x,y)# penalty ? +np.sum(np.diff(sac_flow_adjustment)**2)

## Focus on a time period, e.g. 1990-09 to 1990-10
Choose a time window and then create an inputs data set that includes 117 days, i.e. the antecedent memory of the system
Then choose the initial value xvar ( represents additional flow required in 'sac' )

In [ ]:
twslice=slice('1990-09-01','1990-09-30')
var_start=pd.to_datetime(twslice.start)-pd.Timedelta(117,'D')
var_end=pd.to_datetime(twslice.stop)
# increase exports by 100 only for time slice
dfx0=dfin_on.copy()
dfx0=dfx0.loc[var_start:var_end,:]
dfx0.loc[twslice,'exports']+=100
#initial guess same as amount of exports (100) above (array size is +1 as end is included)
xvar=np.zeros((var_end-var_start).days+1)
xvar[-30:]=100

In [ ]:
dfx0.loc[twslice,['sac','exports']].hvplot()*dfin_on.loc[twslice,['sac','exports']].hvplot()

## Choose SLSQP optimizer for the cost function
The cost function is the SSE w.r.t to the EC before the exports were increased. This should give us the additional 'sac' flow needed to minimize the impact of additional exports. In the Sacramento San Joaquin Delta this is can be used to calculated "carriage" water i.e. the additional water above exports increase (in this case 100) to have no impact to the water quality conditions

In [ ]:
xguess=xvar
opt_result_slsqp=optimize.minimize(cost_func,xguess,(model,dfx0,dfp),method='SLSQP',
                              options={'ftol': 1e-2, 'eps':1, 'maxiter':50})#,callback=optimize_callback)
opt_result_slsqp

In [ ]:
dfans=adjust_input(opt_result_slsqp.x,'sac',dfx0)
dfin_on.loc[var_start:var_end,['sac','exports']].hvplot(label='base')\
 *dfx0.loc[var_start:var_end,['sac','exports']].hvplot(label='exports+100')\
 *dfans.loc[var_start:var_end,['sac','exports']].hvplot(label='sac adjusted')

In [ ]:
pd.DataFrame(opt_result_slsqp.x,dfx0.index).hvplot(label='Additional Sacramento Flow (cfs) required to offset impact of +100cfs exports')

In [ ]:
(pd.DataFrame(opt_result_slsqp.x,dfx0.index)-100)[twslice].hvplot(label='Carriage water for 100 cfs exports increase')

In [ ]:
dfadj=adjust_predict(model,opt_result_slsqp.x,'sac',dfx0)
dfexp=adjust_predict(model,0,'sac',dfx0)

In [ ]:
dfadj.hvplot(label='sac adjusted')*dfp.loc[dfadj.index].hvplot(label='base')*dfexp.loc[dfadj.index].hvplot(label='exp+100')

In [ ]:
df1=pd.concat([model.predict(dfin_on),model.predict(dfx0)],axis=1)
df1.columns=['base','exports']
df1[var_start:var_end].hvplot(title='EC')

In [ ]:
opt_result_cobyla=optimize.minimize(cost_func,xvar,(model,dfx0,dfp),method='cobyla',options={'maxiter':1000,'tol':1e-2})

In [ ]:
opt_result_cobyla

In [ ]:
pd.DataFrame(opt_result_slsqp.x,dfx0.index).hvplot(label='slsqp')*pd.DataFrame(opt_result_cobyla.x,dfx0.index).hvplot(label='cobyla')

In [ ]:
#opt_result_bfgs=optimize.minimize(cost_func,xvar,(model,dfx0,dfp),method='bfgs',options={'eps':1, 'gtol':1, 'maxiter':50})